In [91]:
import berserk
import time
import signal
import threading

In [30]:
API_TOKEN = "lip_Uvmcg05EwH4ZWCxMHSkb"
GAME_ID = ""

In [31]:
session = berserk.TokenSession(API_TOKEN)
client = berserk.Client(session=session)

In [32]:
# game = berserk.clients.Challenges(session).create_ai(level=1)

In [33]:
# GAME_ID = game["id"]
#print(game["id"])

In [36]:
# board = berserk.clients.Board(session)
# board.make_move(GAME_ID, "e2e4")
ongoing_games = berserk.clients.Games(session)
game = ongoing_games.get_ongoing()[0]

{'type': 'gameState', 'moves': 'e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4 c2c3 h7h5 g1f3 c8e6 c1f4 e6g4 e1g1 d4d3 b1a3 f8e7 h2h3 g4e6 a3b5 e8f8 b5d6', 'wtime': datetime.datetime(1970, 1, 25, 20, 31, 23, 647000, tzinfo=datetime.timezone.utc), 'btime': datetime.datetime(1970, 1, 25, 20, 31, 23, 647000, tzinfo=datetime.timezone.utc), 'winc': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'binc': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'status': 'started'}
{'type': 'gameState', 'moves': 'e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4 c2c3 h7h5 g1f3 c8e6 c1f4 e6g4 e1g1 d4d3 b1a3 f8e7 h2h3 g4e6 a3b5 e8f8 b5d6 c6e5', 'wtime': datetime.datetime(1970, 1, 25, 20, 31, 23, 647000, tzinfo=datetime.timezone.utc), 'btime': datetime.datetime(1970, 1, 25, 20, 31, 23, 647000, tzinfo=datetime.timezone.utc), 'winc': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'binc': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezon

In [90]:
# Try timeout 
generator = berserk.clients.Board(session).stream_game_state(game["gameId"])
print(next(generator))
def get_next_move():
    time.sleep(WAIT_TIME)
    return next(generator)

class Timeout():
    class Timeout(Exception):
        pass
    def __init__(self, sec):
        self.sec = sec

    def __enter__(self):
        signal.signal(signal.SIGALARM, self.raise_timeout)
        signal.alarm(self.sec)
    
    def __exit__(self, *args):
        signal.alarm(0)
    
    def raise_timeout(self, *args):
        raise Timeout.Timeout()
    
for i in range(20):
    try:
        with Timeout(WAIT_TIME):
            print(get_next_move)
    except Timeout.Timeout:
        print("TIMEOUT")


{'id': '7ea4AHXb', 'variant': {'key': 'standard', 'name': 'Standard', 'short': 'Std'}, 'speed': 'correspondence', 'perf': {'name': 'Correspondence'}, 'rated': False, 'createdAt': datetime.datetime(2024, 2, 6, 16, 58, 49, 521000, tzinfo=datetime.timezone.utc), 'white': {'id': 'theseus39', 'name': 'theseus39', 'title': None, 'rating': 1500, 'provisional': True}, 'black': {'aiLevel': 1}, 'initialFen': 'startpos', 'type': 'gameFull', 'state': {'type': 'gameState', 'moves': 'e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4 c2c3 h7h5 g1f3 c8e6 c1f4 e6g4 e1g1 d4d3 b1a3 f8e7 h2h3 g4e6 a3b5 e8f8 b5d6 c6e5 f4e5 g8f6 d6f7 d8e8 f7h8 a8d8 e5c7 f8g8 c7d8 e7c5 d8f6 e6h3 g2h3 e8e4 d1b3 e4c4', 'wtime': 2147483647, 'btime': 2147483647, 'winc': 0, 'binc': 0, 'status': 'started'}}


AttributeError: module 'signal' has no attribute 'SIGALARM'

In [111]:
# Wait that works on 1
generator = berserk.clients.Board(session).stream_game_state(game["gameId"])
next(generator) # Initial

def next_move_w_wait(e, moves):
    next_move = None
    next_move = next(generator)
    if next_move is not None:
        e.set()
        moves.append(next_move)

try:
    berserk.clients.Board(session).make_move(game["gameId"], "g7f6")
except:
    berserk.clients.Board(session).make_move(game["gameId"], "f6g7")
next(generator) # This move


# Now get next move
moves = []
e = threading.Event()
t = threading.Thread(target=next_move_w_wait, args=(e, moves))
t.start()
WAIT_TIME = 5
res = t.join(WAIT_TIME)
if t.is_alive():
    print("killing thread")
    e.set()
else:
    print(moves[-1])


e = threading.Event()
t = threading.Thread(target=next_move_w_wait, args=(e, moves))
t.start()
WAIT_TIME = 5
res = t.join(WAIT_TIME)
if t.is_alive():
    print("killing thread")
    e.set()
else:
    print(moves[-1])

{'type': 'gameState', 'moves': 'e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4 c2c3 h7h5 g1f3 c8e6 c1f4 e6g4 e1g1 d4d3 b1a3 f8e7 h2h3 g4e6 a3b5 e8f8 b5d6 c6e5 f4e5 g8f6 d6f7 d8e8 f7h8 a8d8 e5c7 f8g8 c7d8 e7c5 d8f6 e6h3 g2h3 e8e4 d1b3 e4c4 b3c4 g8f8 c4c5 f8e8 f6g7 e8d7 c5d5 d7e8 g7f6 b7b6 f6g7 b6b5 g7f6 a7a6 f6g7 a6a5 g7f6 h5h4 f6g7 b5b4 g7f6 b4c3 b2c3 e8f8 f6g7 f8e7 g7f6 e7f6', 'wtime': datetime.datetime(1970, 1, 25, 20, 31, 23, 647000, tzinfo=datetime.timezone.utc), 'btime': datetime.datetime(1970, 1, 25, 20, 31, 23, 647000, tzinfo=datetime.timezone.utc), 'winc': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'binc': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'status': 'started'}
killing thread


In [112]:
generator = berserk.clients.Board(session).stream_game_state("251xrUYZ")
next(generator) # Initial

{'id': '7ea4AHXb',
 'variant': {'key': 'standard', 'name': 'Standard', 'short': 'Std'},
 'speed': 'correspondence',
 'perf': {'name': 'Correspondence'},
 'rated': False,
 'createdAt': datetime.datetime(2024, 2, 6, 16, 58, 49, 521000, tzinfo=datetime.timezone.utc),
 'white': {'id': 'theseus39',
  'name': 'theseus39',
  'title': None,
  'rating': 1500,
  'provisional': True},
 'black': {'aiLevel': 1},
 'initialFen': 'startpos',
 'type': 'gameFull',
 'state': {'type': 'gameState',
  'moves': 'e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4 c2c3 h7h5 g1f3 c8e6 c1f4 e6g4 e1g1 d4d3 b1a3 f8e7 h2h3 g4e6 a3b5 e8f8 b5d6 c6e5 f4e5 g8f6 d6f7 d8e8 f7h8 a8d8 e5c7 f8g8 c7d8 e7c5 d8f6 e6h3 g2h3 e8e4 d1b3 e4c4 b3c4 g8f8 c4c5 f8e8 f6g7 e8d7 c5d5 d7e8 g7f6 b7b6 f6g7 b6b5 g7f6 a7a6 f6g7 a6a5 g7f6 h5h4 f6g7 b5b4 g7f6 b4c3 b2c3 e8f8 f6g7 f8e7 g7f6 e7f6 d5e5',
  'wtime': 2147483647,
  'btime': 2147483647,
  'winc': 0,
  'binc': 0,
  'status': 'mate',
  'winner': 'white'}}

In [113]:
generator = berserk.clients.Board(session).stream_game_state("251xrUYZ")
print(next(generator)) # Initial

{'id': '251xrUYZ', 'variant': {'key': 'standard', 'name': 'Standard', 'short': 'Std'}, 'speed': 'correspondence', 'perf': {'name': 'Correspondence'}, 'rated': False, 'createdAt': datetime.datetime(2024, 2, 6, 21, 8, 41, 718000, tzinfo=datetime.timezone.utc), 'white': {'aiLevel': 3}, 'black': {'id': 'theseus39', 'name': 'theseus39', 'title': None, 'rating': 1500, 'provisional': True}, 'initialFen': 'startpos', 'type': 'gameFull', 'state': {'type': 'gameState', 'moves': 'e2e4', 'wtime': 2147483647, 'btime': 2147483647, 'winc': 0, 'binc': 0, 'status': 'started'}}


In [114]:
games = berserk.clients.Games(session).get_ongoing()
game = None
for g in games:
    if g["gameId"] == "251xrUYZ":
        game = g
print(game)

{'fullId': '251xrUYZHJt7', 'gameId': '251xrUYZ', 'fen': 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1', 'color': 'black', 'lastMove': 'e2e4', 'source': 'ai', 'status': {'id': 20, 'name': 'started'}, 'variant': {'key': 'standard', 'name': 'Standard'}, 'speed': 'correspondence', 'perf': 'correspondence', 'rated': False, 'hasMoved': False, 'opponent': {'id': None, 'username': 'Stockfish level 3', 'ai': 3}, 'isMyTurn': True}


In [106]:
WAIT_TIME = 5
class MoveStream:
    def __init__(self, generator):
        self.generator = generator
        self.initial_state = next(self.generator)
    
    def get_next_helper(self, e, next_move):
        next_move = next(self.generator)
        if next_move is not None:
            e.set()
            print(next_move)
        return next_move
    
    def print_initial_state(self):
        print(self.initial_state)


    def get_next_move(self):
        e = threading.Event()
        t = threading.Thread(target=self.get_next_helper, args=(e, None))
        t.start()
        res = t.join(WAIT_TIME)
        if t.is_alive():
            print("killing thread")
            e.set()
        else:
            print("t finished")
            print(res)

myMS = MoveStream(berserk.clients.Board(session).stream_game_state(game["gameId"]))
try:
    berserk.clients.Board(session).make_move(game["gameId"], "g7f6")
except:
    berserk.clients.Board(session).make_move(game["gameId"], "f6g7")
print(myMS.get_next_move())
print(myMS.get_next_move())


{'type': 'gameState', 'moves': 'e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4 c2c3 h7h5 g1f3 c8e6 c1f4 e6g4 e1g1 d4d3 b1a3 f8e7 h2h3 g4e6 a3b5 e8f8 b5d6 c6e5 f4e5 g8f6 d6f7 d8e8 f7h8 a8d8 e5c7 f8g8 c7d8 e7c5 d8f6 e6h3 g2h3 e8e4 d1b3 e4c4 b3c4 g8f8 c4c5 f8e8 f6g7 e8d7 c5d5 d7e8 g7f6 b7b6 f6g7 b6b5 g7f6 a7a6 f6g7', 'wtime': datetime.datetime(1970, 1, 25, 20, 31, 23, 647000, tzinfo=datetime.timezone.utc), 'btime': datetime.datetime(1970, 1, 25, 20, 31, 23, 647000, tzinfo=datetime.timezone.utc), 'winc': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'binc': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), 'status': 'started'}
t finished
None
None
{'type': 'gameState', 'moves': 'e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4 c2c3 h7h5 g1f3 c8e6 c1f4 e6g4 e1g1 d4d3 b1a3 f8e7 h2h3 g4e6 a3b5 e8f8 b5d6 c6e5 f4e5 g8f6 d6f7 d8e8 f7h8 a8d8 e5c7 f8g8 c7d8 e7c5 d8f6 e6h3 g2h3 e8e4 d1b3 e4c4 b3c4 g8f8 c4c5 f8e8 f6g7 e8d7 c5d5 d7e8 g7f6 b7b6 f6g7 b6b5 g7f6 

In [85]:
for i in range(20):
    print(myMS.get_next())

StopIteration: 

In [77]:
move_stream = berserk.clients.Board(session).stream_game_state(game["gameId"])
print(next(move_stream)['state']['moves'])

# Making a move is just "startsquare_endsquare" ex "e2e4"
berserk.clients.Board(session).make_move(game["gameId"], "h2h3")
print(next(move_stream)['moves'])
#time.sleep(3)
await move_stream.__next__()
print(next(move_stream)['moves'])



e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4 c2c3 h7h5 g1f3 c8e6 c1f4 e6g4 e1g1 d4d3 b1a3 f8e7
e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4 c2c3 h7h5 g1f3 c8e6 c1f4 e6g4 e1g1 d4d3 b1a3 f8e7 h2h3


TypeError: object dict can't be used in 'await' expression

In [76]:
move_stream = move_stream = berserk.clients.Board(session).stream_game_state(game["gameId"])
print("Initial State: " + str(next(move_stream)))

print(next(move_stream))


Initial State: {'id': '7ea4AHXb', 'variant': {'key': 'standard', 'name': 'Standard', 'short': 'Std'}, 'speed': 'correspondence', 'perf': {'name': 'Correspondence'}, 'rated': False, 'createdAt': datetime.datetime(2024, 2, 6, 16, 58, 49, 521000, tzinfo=datetime.timezone.utc), 'white': {'id': 'theseus39', 'name': 'theseus39', 'title': None, 'rating': 1500, 'provisional': True}, 'black': {'aiLevel': 1}, 'initialFen': 'startpos', 'type': 'gameFull', 'state': {'type': 'gameState', 'moves': 'e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4 c2c3 h7h5 g1f3 c8e6 c1f4 e6g4 e1g1 d4d3 b1a3 f8e7', 'wtime': 2147483647, 'btime': 2147483647, 'winc': 0, 'binc': 0, 'status': 'started'}}


KeyboardInterrupt: 

In [67]:
# This is the closest thing to working
# First thing in generator is game data, then gameStates
move_stream = berserk.clients.Board(session).stream_game_state(game["gameId"])
print(next(move_stream)['state']['moves'])

e2e4 e7e5 d2d4 d7d5 f1b5 c7c6 b5c6 b8c6 d4e5 d5d4


In [46]:
moves = berserk.clients.Board(session).stream_game_state(game["gameId"])
curr_move = next(moves)
while curr_move:
    print(curr_move)
    curr_move = next(moves)
# for move in moves:
#     print(move)

{'id': '7ea4AHXb', 'variant': {'key': 'standard', 'name': 'Standard', 'short': 'Std'}, 'speed': 'correspondence', 'perf': {'name': 'Correspondence'}, 'rated': False, 'createdAt': datetime.datetime(2024, 2, 6, 16, 58, 49, 521000, tzinfo=datetime.timezone.utc), 'white': {'id': 'theseus39', 'name': 'theseus39', 'title': None, 'rating': 1500, 'provisional': True}, 'black': {'aiLevel': 1}, 'initialFen': 'startpos', 'type': 'gameFull', 'state': {'type': 'gameState', 'moves': 'e2e4 e7e5', 'wtime': 2147483647, 'btime': 2147483647, 'winc': 0, 'binc': 0, 'status': 'started'}}


KeyboardInterrupt: 